In [1]:
pip install openai langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_openai import AzureOpenAIEmbeddings
import os
os.environ["AZURE_OPENAI_API_KEY"] = "Type Your OPENAI API key here"
os.environ["AZURE_OPENAI_ENDPOINT"] = "Type your OPENAI ENDPOINT here"

embedding = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002"
)



In [3]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [4]:
import requests
from langchain_openai import AzureOpenAIEmbeddings

# Test the connection first
azure_endpoint = "Type your OPENAI ENDPOINT here"  # Replace with your endpoint
api_key = "Type Your OPENAI API key here"  # Replace with your key

# Add error handling and connection test
try:
    # Initialize with timeout parameter
    embedding = AzureOpenAIEmbeddings(
        azure_deployment="text-embedding-ada-002",
        azure_endpoint=azure_endpoint,
        api_key=api_key,
        timeout=30  # Add timeout in seconds
    )
    
    # Test with a simple query
    test_embedding = embedding.embed_query("Test connection")
    print("Connection successful!")
    
except Exception as e:
    print(f"Connection error: {str(e)}")

Connection successful!


In [5]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [6]:
import numpy as np

In [7]:
np.dot(embedding1, embedding2)

0.9629478572630695

In [8]:
np.dot(embedding1, embedding3)

0.7700566174953781

In [9]:
np.dot(embedding2, embedding3)

0.7588785743809351

In [11]:
persist_directory = 'docs/chroma/'

In [12]:
import shutil
import os

# Method 1: Using shutil
try:
    shutil.rmtree('./docs/chroma')
    print("Successfully removed old database files")
except FileNotFoundError:
    print("No existing database found")

# Method 2: Using os
try:
    os.system('rm -rf ./docs/chroma')
    print("Successfully removed old database files")
except Exception as e:
    print(f"Error: {e}")

# Method 3: Check if exists first
if os.path.exists('./docs/chroma'):
    shutil.rmtree('./docs/chroma')
    print("Successfully removed old database files")
else:
    print("No existing database found")

No existing database found
Successfully removed old database files
No existing database found


In [16]:
!pip install openai==1.6.1
!pip install langchain==0.1.6
!pip install langchain-community==0.0.16
!pip install langchain-openai==0.0.5
!pip install pypdf
!pip install numpy

  Using cached langchain_community-0.0.16-py3-none-any.whl.metadata (7.8 kB)
Using cached langchain_community-0.0.16-py3-none-any.whl (1.6 MB)
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.20
    Uninstalling langchain-community-0.0.20:
      Successfully uninstalled langchain-community-0.0.20


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.6 requires langchain-community<0.1,>=0.0.18, but you have langchain-community 0.0.16 which is incompatible.


  Using cached openai-1.52.1-py3-none-any.whl.metadata (24 kB)
Using cached openai-1.52.1-py3-none-any.whl (386 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.6.1
    Uninstalling openai-1.6.1:
      Successfully uninstalled openai-1.6.1


In [17]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pypdf import PdfReader
import numpy as np
from typing import List, Tuple
import os
import json

class SimpleVectorStore:
    def __init__(self, embeddings):
        self.embeddings = embeddings
        self.texts = []
        self.vectors = []
    
    def add_texts(self, texts: List[str]):
        print(f"Processing {len(texts)} text chunks...")
        for i, text in enumerate(texts, 1):
            vector = self.embeddings.embed_query(text)
            self.vectors.append(vector)
            self.texts.append(text)
            if i % 10 == 0:
                print(f"Processed {i}/{len(texts)} chunks")
    
    def similarity_search(self, query: str, k: int = 4) -> List[Tuple[str, float]]:
        query_vector = self.embeddings.embed_query(query)
        similarities = [np.dot(query_vector, vec) / (np.linalg.norm(query_vector) * np.linalg.norm(vec)) 
                       for vec in self.vectors]
        
        sorted_indices = np.argsort(similarities)[::-1][:k]
        return [(self.texts[i], similarities[i]) for i in sorted_indices]
    
    def save(self, directory: str):
        os.makedirs(directory, exist_ok=True)
        data = {
            'texts': self.texts,
            'vectors': [v.tolist() for v in self.vectors]
        }
        with open(os.path.join(directory, 'vector_store.json'), 'w') as f:
            json.dump(data, f)
    
    def load(self, directory: str):
        with open(os.path.join(directory, 'vector_store.json'), 'r') as f:
            data = json.load(f)
        self.texts = data['texts']
        self.vectors = [np.array(v) for v in data['vectors']]

In [20]:
# Initialize embeddings
print("Initializing Azure OpenAI embeddings...")
embeddings = AzureOpenAIEmbeddings(
    api_key="Type your OPENAI API key here",          # Your existing API key
    azure_endpoint="Type your OPENAI ENDPOINT here",    # Your existing endpoint
    deployment="text-embedding-ada-002",    # Use the deployment name you created for embeddings
    api_version="2023-05-15"
)

# Test embeddings
test_text = "Testing Azure OpenAI connection"
try:
    test_embedding = embeddings.embed_query(test_text)
    print("✓ Azure OpenAI connection successful!")
    print(f"✓ Embedding dimension: {len(test_embedding)}")
except Exception as e:
    print(f"❌ Error connecting to Azure OpenAI: {str(e)}")
    print("\nPlease check:")
    print("1. API key is correct")
    print("2. Endpoint URL is correct")
    print("3. You have created a deployment using 'text-embedding-ada-002' model")

Initializing Azure OpenAI embeddings...
✓ Azure OpenAI connection successful!
✓ Embedding dimension: 1536


In [21]:
import numpy as np
from typing import List, Tuple
import os
import json

class SimpleVectorStore:
    def __init__(self, embeddings):
        self.embeddings = embeddings
        self.texts = []
        self.vectors = []
    
    def add_texts(self, texts: List[str]):
        print(f"Processing {len(texts)} text chunks...")
        for i, text in enumerate(texts, 1):
            vector = self.embeddings.embed_query(text)
            self.vectors.append(vector)
            self.texts.append(text)
            if i % 10 == 0:
                print(f"Processed {i}/{len(texts)} chunks")
    
    def similarity_search(self, query: str, k: int = 4) -> List[Tuple[str, float]]:
        query_vector = self.embeddings.embed_query(query)
        similarities = [np.dot(query_vector, vec) / (np.linalg.norm(query_vector) * np.linalg.norm(vec)) 
                       for vec in self.vectors]
        
        sorted_indices = np.argsort(similarities)[::-1][:k]
        return [(self.texts[i], similarities[i]) for i in sorted_indices]

In [22]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pypdf import PdfReader

# Initialize embeddings (use your working configuration)
embeddings = AzureOpenAIEmbeddings(
    api_key="Type your OPENAI API key here",          
    azure_endpoint="Type your OPENAI ENDPOINT here",    
    deployment="text-embedding-ada-002",    
    api_version="2023-05-15"
)

# Read PDF
print("Reading PDF...")
pdf_path = "MachineLearning-Lecture01.pdf"  # Make sure this path is correct
pdf = PdfReader(pdf_path)

# Extract text
text = ""
for page_num, page in enumerate(pdf.pages, 1):
    text += page.extract_text()
    print(f"Processed page {page_num}/{len(pdf.pages)}")
print("✓ PDF text extracted")

# Split text
print("\nSplitting text...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)
texts = text_splitter.split_text(text)
print(f"✓ Split into {len(texts)} chunks")

# Create vector store
print("\nCreating vector store...")
vectorstore = SimpleVectorStore(embeddings)
vectorstore.add_texts(texts)
print("✓ Vector store created")

Reading PDF...
Processed page 1/22
Processed page 2/22
Processed page 3/22
Processed page 4/22
Processed page 5/22
Processed page 6/22
Processed page 7/22
Processed page 8/22
Processed page 9/22
Processed page 10/22
Processed page 11/22
Processed page 12/22
Processed page 13/22
Processed page 14/22
Processed page 15/22
Processed page 16/22
Processed page 17/22
Processed page 18/22
Processed page 19/22
Processed page 20/22
Processed page 21/22
Processed page 22/22
✓ PDF text extracted

Splitting text...
✓ Split into 70 chunks

Creating vector store...
Processing 70 text chunks...
Processed 10/70 chunks
Processed 20/70 chunks
Processed 30/70 chunks
Processed 40/70 chunks
Processed 50/70 chunks
Processed 60/70 chunks
Processed 70/70 chunks
✓ Vector store created
